In [1]:
import numpy as np
from IPython.display import display
import pandas as pd
import itertools

# Getting Data from InstanceX.txt files

In [2]:
def find_horizon(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            if not line.startswith("#") and not line.startswith("SECTION_HORIZON"):
                n_days = int(line)
                break
    n_weekends = int(n_days/7)
    days = np.arange(1, n_days+1, dtype = int)
    weekends = np.arange(1, n_weekends+1, dtype = int)

    return n_days, days, n_weekends, weekends


In [3]:
def find_nurseIDs(file_path):
    NurseIDs = []
    inside_block = False
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_STAFF"):
                inside_block = True
            elif line.startswith("SECTION_DAYS_OFF"):
                inside_block = False
            elif inside_block:
                if line.split(',')[0] not in ['# ID', '']:
                    NurseIDs.append(line.split(',')[0])
    
    return NurseIDs

def find_shiftInfo(file_path):
    ShiftIDs = []
    Shift_lengths = []
    Forbidden_shifts = []
    inside_block = False
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_SHIFTS"):
                inside_block = True
            elif line.startswith("SECTION_STAFF"):
                inside_block = False
            elif inside_block:
                entries = line.split(',')
                if entries[0] not in ['# ShiftID', '']:
                    ShiftIDs.append(entries[0])
                    Shift_lengths.append(entries[1])
                    Forbidden_shifts.append(entries[2].split("|"))
    
    ForbiddenShifts = []
    for i, shift_list in enumerate(Forbidden_shifts):
        if '' in shift_list: shift_list.remove('')
        while len(shift_list) < len(ShiftIDs):
            shift_list.append('NA')
        ForbiddenShifts.append(shift_list)
            


    return ShiftIDs, np.array(Shift_lengths, dtype=int), ForbiddenShifts


In [4]:
def find_MaxShifts(file_path):
    num_shifts = len(find_shiftInfo(file_path)[0])
    MaxShifts = []
    inside_maxshift_block = False
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_STAFF"):
                inside_maxshift_block = True
            elif line.startswith("SECTION_DAYS_OFF"):
                inside_maxshift_block = False
            elif inside_maxshift_block:
                if line.split(',')[0] not in ['# ID', '']:
                    max_shifts = []
                    maxshift = line.split(',')[1]
                    entries = maxshift.split('|')
                    for shift in entries:
                        max_val = shift.split('=')[1]
                        max_shifts.append(int(max_val))
                    MaxShifts.append(max_shifts)
    
    return MaxShifts


def find_MaxMinInfo(file_path):
    MaxTotalMinutes = []
    MinTotalMinutes = []
    MaxConsecutiveShifts = []
    MinConsecutiveShifts = []
    MinConsecutiveDaysOff = []
    MaxWeekends = []
    inside_block = False
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_STAFF"):
                inside_block = True
            elif line.startswith("SECTION_DAYS_OFF"):
                inside_block = False
            elif inside_block:
                if line.split(',')[0] not in ['# ID', '']:
                    entries = line.split(',')
                    MaxTotalMinutes.append(entries[2])
                    MinTotalMinutes.append(entries[3])
                    MaxConsecutiveShifts.append(entries[4])
                    MinConsecutiveShifts.append(entries[5])
                    MinConsecutiveDaysOff.append(entries[6])
                    MaxWeekends.append(entries[7])
    
    return [MaxTotalMinutes, MinTotalMinutes,
    MaxConsecutiveShifts, MinConsecutiveShifts,
    MinConsecutiveDaysOff, MaxWeekends]


In [5]:
def find_DaysOff(file_path):
    DaysOff = []
    inside_block = False
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_DAYS_OFF"):
                inside_block = True
            elif line.startswith("SECTION_SHIFT_ON_REQUESTS"):
                inside_block = False
            elif inside_block:
                if line.split(',')[0] not in ['# EmployeeID', '']:
                    entries = line.split(',')
                    DaysOff.append(np.array(entries[1:], dtype=int)+1)
    return np.array(DaysOff)


In [6]:
def find_NotAssignedPreferredShiftPenalty(file_path):
    # Initialize variables
    shift_on_lines_to_save = []
    inside_on_block = False
    NurseIDs = find_nurseIDs(file_path)
    ShiftIDs = find_shiftInfo(file_path)[0]
    ndays = find_horizon(file_path)[0]
    Shift_On_Requests = []

    # Open the file for reading
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_SHIFT_ON_REQUESTS"):
                inside_on_block = True
            elif line.startswith("SECTION_SHIFT_OFF_REQUESTS"):
                inside_on_block = False
            elif inside_on_block:
                shift_on_lines_to_save.append(line)
    
    for i in NurseIDs:
        matrix = np.zeros((ndays, len(ShiftIDs)), dtype = int)
        for line in shift_on_lines_to_save[1:-1]:
            x = line.split(',')
            if x[0] == i:
                matrix[int(x[1]), int(ShiftIDs.index(x[2]))] = x[3]
        Shift_On_Requests.append(matrix.tolist())

    return np.array(Shift_On_Requests, dtype = int)



def find_AssignedNotPreferredShiftPenalty(file_path):
    # Initialize variables
    shift_off_lines_to_save = []
    inside_off_block = False
    NurseIDs = find_nurseIDs(file_path)
    ShiftIDs = find_shiftInfo(file_path)[0]
    ndays = find_horizon(file_path)[0]
    Shift_Off_Requests = []
    
    # Open the file for reading
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_SHIFT_OFF_REQUESTS"):
                inside_off_block = True
            elif line.startswith("SECTION_COVER"):
                inside_off_block = False
            elif inside_off_block:
                shift_off_lines_to_save.append(line)
    
    for i in NurseIDs:
        matrix = np.zeros((ndays, len(ShiftIDs)), dtype = int)
        for line in shift_off_lines_to_save[1:-1]:
            x = line.split(',')
            if x[0] == i:
                matrix[int(x[1]), int(ShiftIDs.index(x[2]))] = x[3]
        Shift_Off_Requests.append(matrix.tolist())

    return np.array(Shift_Off_Requests, dtype = int)

In [7]:
def find_CoverageRequirements(file_path):
    ShiftIDs = find_shiftInfo(file_path)[0]
    ndays, days = find_horizon(file_path)[:2]
    inside_block = False
    PreferredCoverage = np.zeros((ndays, len(ShiftIDs)), dtype = int)
    UnderstaffedPenalty = np.zeros((ndays, len(ShiftIDs)), dtype = int)
    OverstaffedPenalty = np.zeros((ndays, len(ShiftIDs)), dtype = int)

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_COVER"):
                inside_block = True
            # elif line.startswith("SECTION_SHIFT_OFF_REQUESTS"):
            #     inside_on_block = False
            elif inside_block:
                if line.split(',')[0] not in ["# Day", ""]:
                    entries = line.split(',')
                    row = int(entries[0])
                    col = int(ShiftIDs.index(entries[1]))
                    PreferredCoverage[row, col] = entries[2]
                    UnderstaffedPenalty[row, col] = entries[3]
                    OverstaffedPenalty[row,col] = entries[4]
    
    return PreferredCoverage, UnderstaffedPenalty, OverstaffedPenalty
            

    

# Choosing a dataset to convert .txt to required .dat file for express

In [8]:
instance_num = input("Choose a file number (between 1 and 24): ")
file_path = f'Instance_txts/Instance{instance_num}.txt'
print("You chose file: ", file_path)

You chose file:  Instance_txts/Instance14.txt


In [9]:
# Convert and store the data from the chosen data instance
NurseIDs = find_nurseIDs(file_path)
ShiftIDs, Shift_lengths, Forbidden_shifts = find_shiftInfo(file_path)
MaxShifts = find_MaxShifts(file_path)
ndays, days, nweekends, weekends = find_horizon(file_path)
DaysOff = find_DaysOff(file_path)
NotAssignedPreferredShiftPenalty = find_NotAssignedPreferredShiftPenalty(file_path)
AssignedNotPreferredShiftPenalty = find_AssignedNotPreferredShiftPenalty(file_path)
PreferredCoverage, UnderstaffedPenalty, OverstaffedPenalty = find_CoverageRequirements(file_path)
MaxTotalMinutes = find_MaxMinInfo(file_path)[0]
MinTotalMinutes = find_MaxMinInfo(file_path)[1]
MaxConsecutiveShifts = find_MaxMinInfo(file_path)[2]
MinConsecutiveShifts = find_MaxMinInfo(file_path)[3]
MinConsecutiveDaysOff = find_MaxMinInfo(file_path)[4]
MaxWeekends = find_MaxMinInfo(file_path)[5]


# Writing the .dat file

In [10]:
# Strings to be written before each array
# File path
output_file = f"Instance_dats/Instance{instance_num}.dat"

# Writing to the file
with open(output_file, 'w') as file:
    # Write NurseIDs
    file.write("NurseID: [")
    np.savetxt(file, NurseIDs, fmt='%s', delimiter=' ', newline=" ")  # You can adjust fmt and delimiter as needed
    file.write("] \n \n")

    # Write Horizon info
    file.write(f"PlanningHorizon: {ndays}")
    file.write("\n")
    file.write("Days: [")
    np.savetxt(file, days, fmt='%d', delimiter=' ', newline=" ")
    file.write("] \n \n")

    file.write(f"NumWeekends: {nweekends}")
    file.write("\n")
    file.write("Weekends: [")
    np.savetxt(file, weekends, fmt='%d', delimiter=' ', newline=" ")
    file.write("] \n \n")

    file.write("\n")

    # Write Shifts info
    file.write("ShiftID: [")
    np.savetxt(file, ShiftIDs, fmt='%s', delimiter=' ', newline=" ")  # You can adjust fmt and delimiter as needed
    file.write("] \n")
    file.write("ShiftLength: [")
    np.savetxt(file, Shift_lengths, fmt='%d', delimiter=' ', newline=" ")
    file.write("] \n")
    file.write("ForbiddenShifts: \n[ \n")
    np.savetxt(file, Forbidden_shifts, fmt='%s', delimiter=' ', newline="\n")  # You can adjust fmt and delimiter as needed
    file.write("] \n")

    file.write("\n")

    # Write days off info
    file.write("DaysOff: \n[")
    for row in DaysOff:
        file.write("[")
        np.savetxt(file, [row], fmt='%d', delimiter=' ', newline=']\n')
    file.write("] \n")

    file.write("\n")

    # Write MaxShifts info
    file.write("MaxShifts: \n[")
    for row in MaxShifts:
        np.savetxt(file, [row], fmt='%d', delimiter=' ', newline='\n')
    file.write("] \n")

    file.write("\n")

    # Write MaxMinInfo
    file.write("MaxTotalMinutes: [")
    np.savetxt(file, MaxTotalMinutes, fmt='%s', delimiter=' ', newline=" ")  
    file.write("] \n")
    file.write("MinTotalMinutes: [")
    np.savetxt(file, MinTotalMinutes, fmt='%s', delimiter=' ', newline=" ")  
    file.write("] \n")
    file.write("MaxConsecutiveShifts: [")
    np.savetxt(file, MaxConsecutiveShifts, fmt='%s', delimiter=' ', newline=" ")  
    file.write("] \n")
    file.write("MinConsecutiveShifts: [")
    np.savetxt(file, MinConsecutiveShifts, fmt='%s', delimiter=' ', newline=" ")  
    file.write("] \n")
    file.write("MinConsecutiveDaysOff: [")
    np.savetxt(file, MinConsecutiveDaysOff, fmt='%s', delimiter=' ', newline=" ")  
    file.write("] \n")
    file.write("MaxWeekends: [")
    np.savetxt(file, MaxWeekends, fmt='%s', delimiter=' ', newline=" ")  
    file.write("] \n")

    file.write("\n")

    # Write Preferred Coverage info
    file.write("PreferredCoverage: \n[")
    for row in PreferredCoverage:
        np.savetxt(file, [row], fmt='%d', delimiter=' ', newline='\n')
    file.write("] \n")

    file.write("\n")

    # Write UnderstaffedPenalty
    file.write("UnderstaffedPenalty: \n[")
    for row in UnderstaffedPenalty:
        np.savetxt(file, [row], fmt='%d', delimiter=' ', newline='\n')
    file.write("] \n")

    file.write("\n")

    # Write UnderstaffedPenalty
    file.write("OverstaffedPenalty: \n[")
    for row in OverstaffedPenalty:
        np.savetxt(file, [row], fmt='%d', delimiter=' ', newline='\n')
    file.write("] \n")

    file.write("\n")

    # Write AssignedNotPreferredPenalty
    file.write("AssignedNotPreferredShiftPenalty: \n[")
    for row in AssignedNotPreferredShiftPenalty:
            np.savetxt(file, row, fmt='%d', delimiter=' ', newline='\n')
    file.write("] \n")

    file.write("\n")

    # Write NotAssignedPreferredPenalty
    file.write("NotAssignedPreferredShiftPenalty: \n[")
    for row in NotAssignedPreferredShiftPenalty:
            np.savetxt(file, row, fmt='%d', delimiter=' ', newline='\n')
    file.write("] \n")


print(f"Arrays saved to {output_file}")

Arrays saved to Instance_dats/Instance14.dat


## Code for defining neighbourhood by single day shuffling

In [11]:
def make_single_day_neighbourhoods(roster, NurseIDs = NurseIDs, 
                                   days = days, ShiftIDs = ShiftIDs, 
                                   Forbidden_shifts=Forbidden_shifts, 
                                   DaysOff=DaysOff, MaxShifts=MaxShifts, 
                                   MaxTotalMinutes=MaxTotalMinutes, 
                                   MinTotalMinutes=MinTotalMinutes, 
                                   MaxConsecutiveShifts=MaxConsecutiveShifts, 
                                   MinConsecutiveShifts=MinConsecutiveShifts, 
                                   MinConsecutiveDaysOff=MinConsecutiveDaysOff,
                                   weekends=weekends, nweekends=nweekends, 
                                   MaxWeekends=MaxWeekends):
    single_day_neighbourhoods = []

    for d in days: # go through each day
        for ind, i in enumerate(NurseIDs): # choose first nurse
            for j in NurseIDs[ind+1:]: # choose second nurse
                neighbourhood = roster.copy()   # neighbourhood to store after swap
                status = "proceed"              # default status is to proceed with a swap unless found to be infeasible

                i_new_shift = roster.loc[j, str(d)]   # shifts for Nurse i post swap
                j_new_shift = roster.loc[i, str(d)]   # shifts for Nurse j post swap

                # update neighbourhood  post swap
                neighbourhood.loc[i, str(d)] = i_new_shift 
                neighbourhood.loc[j, str(d)] = j_new_shift

                # HC2: Check Forbidden shifts constraint
                if i_new_shift != " ":
                    i_FS_next = Forbidden_shifts[ShiftIDs.index(i_new_shift)]
                    
                    if d < days[-1] and roster.loc[i, str(d+1)] in i_FS_next:
                        status = "not feasible swap"

                    if d >= days[1] and roster.loc[i, str(d-1)] != " ":
                        if i_new_shift in Forbidden_shifts[ShiftIDs.index(roster.loc[i, str(d-1)])]:
                            status = "not feasible swap"

                if status == "not feasible swap":
                    # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                    continue 

                if j_new_shift != " ":
                    j_FS_next = Forbidden_shifts[ShiftIDs.index(j_new_shift)]
                    
                    if d < days[-1] and roster.loc[j, str(d+1)] in j_FS_next:
                        status = "not feasible swap"

                    if d >= days[1] and roster.loc[j, str(d-1)] != " ":
                        if j_new_shift in Forbidden_shifts[ShiftIDs.index(roster.loc[j, str(d-1)])]:
                            status = "not feasible swap"

                if status == "not feasible swap":
                    # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                    continue 

                
                # HC8: Check days off constraint
                if i_new_shift != " " and d in DaysOff[NurseIDs.index(i)]:
                    status = "not feasible swap"
                
                if j_new_shift != " " and d in DaysOff[NurseIDs.index(j)]:
                    status = "not feasible swap"
                
                if status == "not feasible swap":
                    # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                    continue 


                # HC3: Check maximum number of shifts of each type constraint
                i_MaxShifts = MaxShifts[NurseIDs.index(i)]
                j_MaxShifts = MaxShifts[NurseIDs.index(j)]
                for shift_ind, shift in enumerate(ShiftIDs):
                    if (neighbourhood.loc[i]==shift).sum() > i_MaxShifts[shift_ind] or (neighbourhood.loc[j]==shift).sum() > j_MaxShifts[shift_ind]:
                        status = "not feasible swap"
                        
                if status == "not feasible swap":
                    # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                    continue 

                    
                # HC4: Check minimum and maximum work times
                i_MaxTotalMinutes = int(MaxTotalMinutes[NurseIDs.index(i)])
                i_MinTotalMinutes = int(MinTotalMinutes[NurseIDs.index(i)])
                j_MaxTotalMinutes = int(MaxTotalMinutes[NurseIDs.index(j)])
                j_MinTotalMinutes = int(MinTotalMinutes[NurseIDs.index(j)])
                
                i_total = 0
                j_total = 0
                for shift_ind, shift in enumerate(ShiftIDs):
                    i_total += (neighbourhood.loc[i]==shift).sum()*Shift_lengths[shift_ind]
                    j_total += (neighbourhood.loc[j]==shift).sum()*Shift_lengths[shift_ind]
                    
                if not (i_MinTotalMinutes <= i_total <= i_MaxTotalMinutes):
                    status = "not feasible swap"

                if not (j_MinTotalMinutes <= j_total <= j_MaxTotalMinutes):
                    status = "not feasible swap"

                if status == "not feasible swap":
                    # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                    continue 
                
                # HC5 and 6: Check minimum + maximum consecutive shifts and minimum consecutive days off
                i_cons_on_counts = []
                i_cons_off_counts = []
                i_cons_on = 0
                i_cons_off = 0

                j_cons_on_counts = []
                j_cons_off_counts = []
                j_cons_on = 0
                j_cons_off = 0

                i_schedule = neighbourhood.loc[i]
                j_schedule = neighbourhood.loc[j]
                for day in days:
                    if (i_schedule.loc[str(day)]) == " ":
                        i_cons_off += 1
                        i_cons_on = 0
                    elif (i_schedule.loc[str(day)]) != " ":
                        i_cons_on += 1
                        i_cons_off = 0

                    if (j_schedule.loc[str(day)]) == " ":
                        j_cons_off += 1
                        j_cons_on = 0
                    elif (j_schedule.loc[str(day)]) != " ":
                        j_cons_on += 1
                        j_cons_off = 0

                    i_cons_on_counts.append(i_cons_on)
                    i_cons_off_counts.append(i_cons_off)
                    j_cons_on_counts.append(j_cons_on)
                    j_cons_off_counts.append(j_cons_off)
                
                
                i_cons_on_list = [len(list(x[1])) for x in itertools.groupby(i_cons_on_counts, lambda x: x == 0) if not x[0]]
                i_cons_off_list = [len(list(x[1])) for x in itertools.groupby(i_cons_off_counts, lambda x: x == 0) if not x[0]]
                j_cons_on_list = [len(list(x[1])) for x in itertools.groupby(j_cons_on_counts, lambda x: x == 0) if not x[0]]
                j_cons_off_list = [len(list(x[1])) for x in itertools.groupby(j_cons_off_counts, lambda x: x == 0) if not x[0]]

                if max(i_cons_on_list) > int(MaxConsecutiveShifts[NurseIDs.index(i)]):
                    status = "not feasible swap"
                
                if max(j_cons_on_list) > int(MaxConsecutiveShifts[NurseIDs.index(j)]):
                    status = "not feasible swap"

                if (True in [s < int(MinConsecutiveShifts[NurseIDs.index(i)]) for s in i_cons_on_list]):
                    status = "not feasible swap"

                if (True in [s < int(MinConsecutiveShifts[NurseIDs.index(j)]) for s in j_cons_on_list]):
                    status = "not feasible swap"

                if (True in [s < int(MinConsecutiveDaysOff[NurseIDs.index(i)]) for s in i_cons_off_list]):
                    status = "not feasible swap"

                if (True in [s < int(MinConsecutiveDaysOff[NurseIDs.index(j)]) for s in j_cons_off_list]):
                    status = "not feasible swap"

                if status == "not feasible swap":
                    # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                    continue 

                # HC7: Max weekends constraints
                weekend_days = np.sort(np.concatenate((weekends*7 - 1, weekends*7), axis = 0))
                i_weekends = np.zeros(nweekends, dtype = int)
                j_weekends = np.zeros(nweekends, dtype = int)

                for w_ind, we in enumerate(weekends):
                        if neighbourhood.loc[i, str(7*we-1)] != " " or neighbourhood.loc[i, str(7*we)] != " ":
                            i_weekends[w_ind] = 1

                        if not (i_weekends[w_ind] <= (neighbourhood.loc[i, [str(7*we-1), str(7*we)]] != " ").sum() <= 2*i_weekends[w_ind]):
                            status = "not feasible swap"

                        
                        if neighbourhood.loc[j, str(7*we-1)] != " " or neighbourhood.loc[j, str(7*we)] != " ":
                            j_weekends[w_ind] = 1

                        if not (j_weekends[w_ind] <= (neighbourhood.loc[j, [str(7*we-1), str(7*we)]] != " ").sum() <= 2*j_weekends[w_ind]):
                            status = "not feasible swap"

                if status == "not feasible swap":
                    # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                    continue          
                    
                if i_weekends.sum() > int(MaxWeekends[NurseIDs.index(i)]) or j_weekends.sum() > int(MaxWeekends[NurseIDs.index(j)]):
                    status = "not feasible swap"

                if status == "not feasible swap":
                    # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                    continue 

                # If all constraints are satisfied after the swap, add this neighbourhood to the list of possible neighbourhoods
                if status == "proceed":
                    single_day_neighbourhoods.append(neighbourhood)

    print("Single day neighbourhoods complete. Number of possible nighbourhoods: ", len(single_day_neighbourhoods))
    return single_day_neighbourhoods
            

            

In [12]:
def make_all_days_neighbourhoods(roster, NurseIDs = NurseIDs, 
                                   days = days, ShiftIDs = ShiftIDs, 
                                   DaysOff=DaysOff, MaxShifts=MaxShifts, 
                                   MaxTotalMinutes=MaxTotalMinutes, 
                                   MinTotalMinutes=MinTotalMinutes, 
                                   MaxConsecutiveShifts=MaxConsecutiveShifts, 
                                   MinConsecutiveShifts=MinConsecutiveShifts, 
                                   MinConsecutiveDaysOff=MinConsecutiveDaysOff,
                                   weekends=weekends, nweekends=nweekends, 
                                   MaxWeekends=MaxWeekends):
    all_days_neighbourhoods = []

    for ind, i in enumerate(NurseIDs): # choose first nurse
        for j in NurseIDs[ind+1:]: # choose second nurse
            neighbourhood = roster.copy()   # neighbourhood to store after swap
            status = "proceed"              # default status is to proceed with a swap unless found to be infeasible

            i_new_shift = roster.loc[j, :]   # shifts for Nurse i post swap
            j_new_shift = roster.loc[i, :]   # shifts for Nurse j post swap

            # update neighbourhood  post swap
            neighbourhood.loc[i, :] = i_new_shift 
            neighbourhood.loc[j, :] = j_new_shift

            # HC1: Check maximum of 1 shift per day constraint is not required as it is satisfied by each nurse in the initial solution
            # HC2: Check Forbidden shifts constraint is not required as it is satisfied by each nurse in the initial solution
            
            # HC8: Check days off constraint
            for d in days:
                if i_new_shift[d-1] != " " and d in DaysOff[NurseIDs.index(i)]:
                    status = "not feasible swap"
                
                if j_new_shift[d-1] != " " and d in DaysOff[NurseIDs.index(j)]:
                    status = "not feasible swap"
                
            if status == "not feasible swap":
                # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                continue 

            
            # HC3: Check maximum number of shifts of each type constraint
            i_MaxShifts = MaxShifts[NurseIDs.index(i)]
            j_MaxShifts = MaxShifts[NurseIDs.index(j)]
            for shift_ind, shift in enumerate(ShiftIDs):
                if (neighbourhood.loc[i]==shift).sum() > i_MaxShifts[shift_ind] or (neighbourhood.loc[j]==shift).sum() > j_MaxShifts[shift_ind]:
                    status = "not feasible swap"
                    
            if status == "not feasible swap":
                # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                continue 

            # HC4: Check minimum and maximum work times
            i_MaxTotalMinutes = int(MaxTotalMinutes[NurseIDs.index(i)])
            i_MinTotalMinutes = int(MinTotalMinutes[NurseIDs.index(i)])
            j_MaxTotalMinutes = int(MaxTotalMinutes[NurseIDs.index(j)])
            j_MinTotalMinutes = int(MinTotalMinutes[NurseIDs.index(j)])
            
            i_total = 0
            j_total = 0
            for shift_ind, shift in enumerate(ShiftIDs):
                i_total += (neighbourhood.loc[i]==shift).sum()*Shift_lengths[shift_ind]
                j_total += (neighbourhood.loc[j]==shift).sum()*Shift_lengths[shift_ind]
                
            if not (i_MinTotalMinutes <= i_total <= i_MaxTotalMinutes):
                status = "not feasible swap"

            if not (j_MinTotalMinutes <= j_total <= j_MaxTotalMinutes):
                status = "not feasible swap"

            if status == "not feasible swap":
                # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                continue 
            
            # HC5 and 6: Check minimum + maximum consecutive shifts and minimum consecutive days off
            i_cons_on_counts = []
            i_cons_off_counts = []
            i_cons_on = 0
            i_cons_off = 0

            j_cons_on_counts = []
            j_cons_off_counts = []
            j_cons_on = 0
            j_cons_off = 0

            i_schedule = neighbourhood.loc[i]
            j_schedule = neighbourhood.loc[j]
            for day in days:
                if (i_schedule.loc[str(day)]) == " ":
                    i_cons_off += 1
                    i_cons_on = 0
                elif (i_schedule.loc[str(day)]) != " ":
                    i_cons_on += 1
                    i_cons_off = 0

                if (j_schedule.loc[str(day)]) == " ":
                    j_cons_off += 1
                    j_cons_on = 0
                elif (j_schedule.loc[str(day)]) != " ":
                    j_cons_on += 1
                    j_cons_off = 0

                i_cons_on_counts.append(i_cons_on)
                i_cons_off_counts.append(i_cons_off)
                j_cons_on_counts.append(j_cons_on)
                j_cons_off_counts.append(j_cons_off)
            
            
            i_cons_on_list = [len(list(x[1])) for x in itertools.groupby(i_cons_on_counts, lambda x: x == 0) if not x[0]]
            i_cons_off_list = [len(list(x[1])) for x in itertools.groupby(i_cons_off_counts, lambda x: x == 0) if not x[0]]
            j_cons_on_list = [len(list(x[1])) for x in itertools.groupby(j_cons_on_counts, lambda x: x == 0) if not x[0]]
            j_cons_off_list = [len(list(x[1])) for x in itertools.groupby(j_cons_off_counts, lambda x: x == 0) if not x[0]]

            if max(i_cons_on_list) > int(MaxConsecutiveShifts[NurseIDs.index(i)]):
                status = "not feasible swap"
            
            if max(j_cons_on_list) > int(MaxConsecutiveShifts[NurseIDs.index(j)]):
                status = "not feasible swap"

            if (True in [s < int(MinConsecutiveShifts[NurseIDs.index(i)]) for s in i_cons_on_list]):
                status = "not feasible swap"

            if (True in [s < int(MinConsecutiveShifts[NurseIDs.index(j)]) for s in j_cons_on_list]):
                status = "not feasible swap"

            if (True in [s < int(MinConsecutiveDaysOff[NurseIDs.index(i)]) for s in i_cons_off_list]):
                status = "not feasible swap"

            if (True in [s < int(MinConsecutiveDaysOff[NurseIDs.index(j)]) for s in j_cons_off_list]):
                status = "not feasible swap"

            if status == "not feasible swap":
                # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                continue 


            # HC7: Max weekends constraints
            weekend_days = np.sort(np.concatenate((weekends*7 - 1, weekends*7), axis = 0))
            i_weekends = np.zeros(nweekends, dtype = int)
            j_weekends = np.zeros(nweekends, dtype = int)

            for w_ind, we in enumerate(weekends):
                    if neighbourhood.loc[i, str(7*we-1)] != " " or neighbourhood.loc[i, str(7*we)] != " ":
                        i_weekends[w_ind] = 1

                    if not (i_weekends[w_ind] <= (neighbourhood.loc[i, [str(7*we-1), str(7*we)]] != " ").sum() <= 2*i_weekends[w_ind]):
                        status = "not feasible swap"

                    
                    if neighbourhood.loc[j, str(7*we-1)] != " " or neighbourhood.loc[j, str(7*we)] != " ":
                        j_weekends[w_ind] = 1

                    if not (j_weekends[w_ind] <= (neighbourhood.loc[j, [str(7*we-1), str(7*we)]] != " ").sum() <= 2*j_weekends[w_ind]):
                        status = "not feasible swap"

            if status == "not feasible swap":
                # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                continue          
                
            if i_weekends.sum() > int(MaxWeekends[NurseIDs.index(i)]) or j_weekends.sum() > int(MaxWeekends[NurseIDs.index(j)]):
                status = "not feasible swap"

            if status == "not feasible swap":
                # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                continue 

            # If all constraints are satisfied after the swap, add this neighbourhood to the list of possible neighbourhoods
            if status == "proceed":
                all_days_neighbourhoods.append(neighbourhood)
    print("All day neighbourhoods complete. Number of possible nighbourhoods: ", len(all_days_neighbourhoods))
    return all_days_neighbourhoods

# Carrying out Variable Neighbourhood Search


In [13]:
def make_multiday_neighbourhoods(roster, NurseIDs = NurseIDs, 
                                   days = days, ShiftIDs = ShiftIDs, 
                                   DaysOff=DaysOff, MaxShifts=MaxShifts, 
                                   MaxTotalMinutes=MaxTotalMinutes, 
                                   MinTotalMinutes=MinTotalMinutes, 
                                   MaxConsecutiveShifts=MaxConsecutiveShifts, 
                                   MinConsecutiveShifts=MinConsecutiveShifts, 
                                   MinConsecutiveDaysOff=MinConsecutiveDaysOff,
                                   weekends=weekends, nweekends=nweekends, 
                                   MaxWeekends=MaxWeekends):
    
    multidays_neighbourhoods = []
    day_sizes = days[1:-1]
    for s in day_sizes:
        for d in days[:len(days) - (s-1)]:
            for ind, i in enumerate(NurseIDs): # choose first nurse
                for j in NurseIDs[ind+1:]: # choose second nurse
                    neighbourhood = roster.copy()   # neighbourhood to store after swap
                    status = "proceed"              # default status is to proceed with a swap unless found to be infeasible

                    i_new_shift = roster.loc[j, str(d):str(d+s-1)]   # shifts for Nurse i post swap
                    j_new_shift = roster.loc[i, str(d):str(d+s-1)]   # shifts for Nurse j post swap

                    # update neighbourhood  post swap
                    neighbourhood.loc[i, str(d):str(d+s-1)] = i_new_shift 
                    neighbourhood.loc[j, str(d):str(d+s-1)] = j_new_shift

                    # HC2: Check Forbidden shifts constraint
                    if i_new_shift[-1] != " ":
                        i_FS_next = Forbidden_shifts[ShiftIDs.index(i_new_shift[-1])]
                        
                        if d+s-1 < days[-1] and roster.loc[i, str(d+s)] in i_FS_next:
                            status = "not feasible swap"

                        if d >= days[1] and roster.loc[i, str(d-1)] != " ":
                            if i_new_shift[0] in Forbidden_shifts[ShiftIDs.index(roster.loc[i, str(d-1)])]:
                                status = "not feasible swap"

                    if status == "not feasible swap":
                        # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                        continue 

                    if j_new_shift[-1] != " ":
                        j_FS_next = Forbidden_shifts[ShiftIDs.index(j_new_shift[-1])]
                        
                        if d+s-1 < days[-1] and roster.loc[j, str(d+s)] in j_FS_next:
                            status = "not feasible swap"

                        if d >= days[1] and roster.loc[j, str(d-1)] != " ":
                            if j_new_shift[0] in Forbidden_shifts[ShiftIDs.index(roster.loc[j, str(d-1)])]:
                                status = "not feasible swap"

                    if status == "not feasible swap":
                        # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                        continue 

                    # HC8: Check days off constraint
                    for day_off in DaysOff[NurseIDs.index(i)]:
                        if day_off in range(d, d+s) and neighbourhood.loc[i, str(day_off)] != " ":
                            status = "not feasible swap"
                    
                    for day_off in DaysOff[NurseIDs.index(j)]:
                        if day_off in range(d, d+s) and neighbourhood.loc[j, str(day_off)] != " ":
                            status = "not feasible swap"
                        
                    if status == "not feasible swap":
                        # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                        continue 

                    # HC3: Check maximum number of shifts of each type constraint
                    i_MaxShifts = MaxShifts[NurseIDs.index(i)]
                    j_MaxShifts = MaxShifts[NurseIDs.index(j)]
                    for shift_ind, shift in enumerate(ShiftIDs):
                        if (neighbourhood.loc[i]==shift).sum() > i_MaxShifts[shift_ind] or (neighbourhood.loc[j]==shift).sum() > j_MaxShifts[shift_ind]:
                            status = "not feasible swap"
                            
                    if status == "not feasible swap":
                        # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                        continue 

                        
                    # HC4: Check minimum and maximum work times
                    i_MaxTotalMinutes = int(MaxTotalMinutes[NurseIDs.index(i)])
                    i_MinTotalMinutes = int(MinTotalMinutes[NurseIDs.index(i)])
                    j_MaxTotalMinutes = int(MaxTotalMinutes[NurseIDs.index(j)])
                    j_MinTotalMinutes = int(MinTotalMinutes[NurseIDs.index(j)])
                    
                    i_total = 0
                    j_total = 0
                    for shift_ind, shift in enumerate(ShiftIDs):
                        i_total += (neighbourhood.loc[i]==shift).sum()*Shift_lengths[shift_ind]
                        j_total += (neighbourhood.loc[j]==shift).sum()*Shift_lengths[shift_ind]
                        
                    if not (i_MinTotalMinutes <= i_total <= i_MaxTotalMinutes):
                        status = "not feasible swap"

                    if not (j_MinTotalMinutes <= j_total <= j_MaxTotalMinutes):
                        status = "not feasible swap"

                    if status == "not feasible swap":
                        # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                        continue 
                    
                    # HC5 and 6: Check minimum + maximum consecutive shifts and minimum consecutive days off
                    i_cons_on_counts = []
                    i_cons_off_counts = []
                    i_cons_on = 0
                    i_cons_off = 0

                    j_cons_on_counts = []
                    j_cons_off_counts = []
                    j_cons_on = 0
                    j_cons_off = 0

                    i_schedule = neighbourhood.loc[i]
                    j_schedule = neighbourhood.loc[j]
                    for day in days:
                        if (i_schedule.loc[str(day)]) == " ":
                            i_cons_off += 1
                            i_cons_on = 0
                        elif (i_schedule.loc[str(day)]) != " ":
                            i_cons_on += 1
                            i_cons_off = 0

                        if (j_schedule.loc[str(day)]) == " ":
                            j_cons_off += 1
                            j_cons_on = 0
                        elif (j_schedule.loc[str(day)]) != " ":
                            j_cons_on += 1
                            j_cons_off = 0

                        i_cons_on_counts.append(i_cons_on)
                        i_cons_off_counts.append(i_cons_off)
                        j_cons_on_counts.append(j_cons_on)
                        j_cons_off_counts.append(j_cons_off)
                    
                    i_cons_on_list = [len(list(x[1])) for x in itertools.groupby(i_cons_on_counts, lambda x: x == 0) if not x[0]]
                    i_cons_off_list = [len(list(x[1])) for x in itertools.groupby(i_cons_off_counts, lambda x: x == 0) if not x[0]]
                    j_cons_on_list = [len(list(x[1])) for x in itertools.groupby(j_cons_on_counts, lambda x: x == 0) if not x[0]]
                    j_cons_off_list = [len(list(x[1])) for x in itertools.groupby(j_cons_off_counts, lambda x: x == 0) if not x[0]]

                    if max(i_cons_on_list) > int(MaxConsecutiveShifts[NurseIDs.index(i)]):
                        status = "not feasible swap"
                    
                    if max(j_cons_on_list) > int(MaxConsecutiveShifts[NurseIDs.index(j)]):
                        status = "not feasible swap"

                    if (True in [s < int(MinConsecutiveShifts[NurseIDs.index(i)]) for s in i_cons_on_list]):
                        status = "not feasible swap"

                    if (True in [s < int(MinConsecutiveShifts[NurseIDs.index(j)]) for s in j_cons_on_list]):
                        status = "not feasible swap"

                    if (True in [s < int(MinConsecutiveDaysOff[NurseIDs.index(i)]) for s in i_cons_off_list]):
                        status = "not feasible swap"

                    if (True in [s < int(MinConsecutiveDaysOff[NurseIDs.index(j)]) for s in j_cons_off_list]):
                        status = "not feasible swap"

                    if status == "not feasible swap":
                        # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                        continue 

                    # HC7: Max weekends constraints
                    weekend_days = np.sort(np.concatenate((weekends*7 - 1, weekends*7), axis = 0))
                    i_weekends = np.zeros(nweekends, dtype = int)
                    j_weekends = np.zeros(nweekends, dtype = int)

                    for w_ind, we in enumerate(weekends):
                            if neighbourhood.loc[i, str(7*we-1)] != " " or neighbourhood.loc[i, str(7*we)] != " ":
                                i_weekends[w_ind] = 1

                            if not (i_weekends[w_ind] <= (neighbourhood.loc[i, [str(7*we-1), str(7*we)]] != " ").sum() <= 2*i_weekends[w_ind]):
                                status = "not feasible swap"

                            
                            if neighbourhood.loc[j, str(7*we-1)] != " " or neighbourhood.loc[j, str(7*we)] != " ":
                                j_weekends[w_ind] = 1

                            if not (j_weekends[w_ind] <= (neighbourhood.loc[j, [str(7*we-1), str(7*we)]] != " ").sum() <= 2*j_weekends[w_ind]):
                                status = "not feasible swap"

                    if status == "not feasible swap":
                        # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                        continue          
                        
                    if i_weekends.sum() > int(MaxWeekends[NurseIDs.index(i)]) or j_weekends.sum() > int(MaxWeekends[NurseIDs.index(j)]):
                        status = "not feasible swap"

                    if status == "not feasible swap":
                        # print(f"I am breaking here at the swap between Nurse {i} and Nurse {j} on day {d}")    
                        continue 

                    # If all constraints are satisfied after the swap, add this neighbourhood to the list of possible neighbourhoods
                    if status == "proceed":
                        multidays_neighbourhoods.append(neighbourhood)

                    if len(multidays_neighbourhoods) >= 10000 and d == len(days)/2:
                        return multidays_neighbourhoods

    
    print("Multi-day neighbourhoods complete. Number of possible neighbourhoods: ", len(multidays_neighbourhoods))
    return multidays_neighbourhoods


## Functions to calculate the objective function and to carry out the Variable Neighbourhood Search algorithm

In [14]:
def calculate_objective(neighbourhood, PreferredCoverage=PreferredCoverage, 
                        AssignedNotPreferredShiftPenalty=AssignedNotPreferredShiftPenalty, 
                        NotAssignedPreferredShiftPenalty=NotAssignedPreferredShiftPenalty):
    objective = 0
    shift_type_penalty_objective = 0
    coverage_penalty_objective = 0

    # Calculate coverage penalties
    for d in days:
        undercoverage_penalty = (UnderstaffedPenalty[d-1]*
                                 np.maximum(np.zeros(PreferredCoverage[d-1].shape, dtype=int), (PreferredCoverage[d-1] - [neighbourhood.loc[:, str(d)][neighbourhood.loc[:, str(d)] == shift].shape[0] for shift in ShiftIDs]))
                                 ).sum()
        
        overcoverage_penalty = (OverstaffedPenalty[d-1]*
                                 np.maximum(np.zeros(PreferredCoverage[d-1].shape, dtype=int), ([neighbourhood.loc[:, str(d)][neighbourhood.loc[:, str(d)] == shift].shape[0] for shift in ShiftIDs] - PreferredCoverage[d-1]))
                                 ).sum()
        
        # print(f"under = {undercoverage_penalty}, over = {overcoverage_penalty}")
        
        coverage_penalty_objective = coverage_penalty_objective + undercoverage_penalty
        coverage_penalty_objective = coverage_penalty_objective + overcoverage_penalty

    # Calculate Shift on/off request penalties
    for i in NurseIDs:
        assgn_not_pref_penalty = ((np.tile(neighbourhood.loc[i,:].T, reps = (1,len(ShiftIDs))
                 ).reshape(ndays,len(ShiftIDs), order = "F") == 
         np.tile(ShiftIDs, reps = (ndays, 1))
         )*AssignedNotPreferredShiftPenalty[NurseIDs.index(i)]).sum()

        not_assgn_pref_penalty = ((np.tile(neighbourhood.loc[i,:].T, reps = (1,len(ShiftIDs))
                 ).reshape(ndays,len(ShiftIDs), order = "F") != 
         np.tile(ShiftIDs, reps = (ndays, 1))
         )*NotAssignedPreferredShiftPenalty[NurseIDs.index(i)]).sum()
        
        # print(f"not assigned preferred = {not_assgn_pref_penalty}, assigned not preferred = {assgn_not_pref_penalty}")
        
        shift_type_penalty_objective = shift_type_penalty_objective + assgn_not_pref_penalty
        shift_type_penalty_objective = shift_type_penalty_objective + not_assgn_pref_penalty

    # Calculate total objective and return this
    objective = coverage_penalty_objective + shift_type_penalty_objective

    return objective
        
        



    

In [15]:
def VNS(initial_roster):
    
    best_roster = initial_roster
    best_objective = calculate_objective(best_roster)
    neighbourhoods = make_single_day_neighbourhoods(initial_roster) + make_all_days_neighbourhoods(roster) + make_multiday_neighbourhoods(roster)
    kmax = len(neighbourhoods)
    print(kmax)
    k = 1
    while k <= kmax:
        new_neighbourhood = neighbourhoods[k-1]
        new_objective = calculate_objective(new_neighbourhood)
        if new_objective < best_objective:
            print(f"New best solution found! Objective = {new_objective}")
            best_roster = new_neighbourhood
            best_objective = new_objective
            k = 1
        else:
            k += 1

    if best_roster.equals(initial_roster):
        print("No better roster found!")
    
    return best_roster



In [16]:
# # Use these to convert the NEOS stuff!!!
to_save_roster_number = input("Give the roster number PLEASE")
roster_to_save = pd.read_csv(f"Roster{to_save_roster_number}.txt", delimiter=",", index_col=0)
roster_to_save.to_csv(f"RosterSolutions/NurseRoster{to_save_roster_number}.csv")





## Convert NurseRosterX.csv to be readable in python

In [17]:
roster_file = f"RosterSolutions/NurseRoster{instance_num}.csv"
roster = pd.read_csv(roster_file, header = 0, delimiter = ",",  index_col=0, skipfooter=4, engine = "python")
roster

,1,2,3,4,5,6,7,8,9,10,...,33,34,35,36,37,38,39,40,41,42
NurseID,,,,,,,,,,,,,,,,,,,,,
A,D,,,D,D,L,N,N,,,...,D,,,L,L,L,L,L,,
B,N,N,,,,D,D,,,N,...,D,D,D,,,D,D,,,
C,E,E,D,N,N,,,E,D,,...,D,,,,L,N,,,,
D,D,D,D,,,L,L,L,L,L,...,L,,,L,L,L,L,L,,
E,L,L,,,,D,D,D,N,N,...,E,D,E,E,N,,,,,
F,D,D,D,D,D,,,D,L,L,...,,L,L,,,,,D,L,
G,L,N,N,,,D,L,L,L,L,...,N,,,L,L,L,L,,,
H,E,E,D,E,E,,,E,E,E,...,,E,E,,,,,,E,E
I,D,D,D,N,N,,,,E,D,...,,,,E,E,D,,,E,N


## Implement VNS

In [18]:
# Calculate the objective of the initial roster
print(f"The objective value of the given roster from the IP solver for instance {instance_num} = \n {calculate_objective(roster)}")

The objective value of the given roster from the IP solver for instance 14 = 
 2795


In [19]:
# # Implement VNS on the initial roster
# new_best_roster = VNS(roster)

In [20]:
# # Save the improved roster from VNS (if any)
# if not new_best_roster.equals(roster):
#     new_best_roster.to_csv(f"RosterSolution_post_VNS/VNS_NurseRoster{instance_num}.csv")